In [37]:
import pandas as pd
import numpy as np
import requests
import json
import math

In [154]:
from datetime import date

In [38]:
price_series = {
    'USA':'CUUR0000SAF116',
    'Houston':'CUURS37BSAF116',
    'DFW':'CUURS37ASAF116'
}

In [121]:
start_year = 1990
end_year = 2022

We can only download 9 years, 11 months of data at a time. If data range is larger than this, then loop over it.

In [122]:
dates = np.arange(start_year, end_year+1)

In [123]:
def download_data(series_ids: list, start_year: int, end_year: int) -> dict:
    assert(isinstance(series_ids, list))
    request_data = json.dumps({"seriesid": series_ids,
                               "startyear":str(start_year),
                               "endyear":str(end_year)}
                             )
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/',
                      headers={'Content-type': 'application/json'},
                      data=request_data
                     )
    return json.loads(p.text)

In [129]:
raw_responses = []
series_ids = list(price_series.values())
if len(dates) > 9:
    dl_date_ranges = np.array_split(dates, np.arange(0,len(dates),9)[1:])
    for dl_dates in dl_date_ranges:
        print('DOWNLOADING:', series_ids, dl_dates[0], dl_dates[-1])
        dl_data = download_data(series_ids, dl_dates[0], dl_dates[-1])
        raw_responses.append(dl_data)
else:
    print('DOWNLOADING:', series_ids, dates[0], dates[-1])
    dl_data = download_data(series_ids, dates[0], dates[-1])
    raw_responses.append(dl_data)

DOWNLOADING: ['CUUR0000SAF116', 'CUURS37BSAF116', 'CUURS37ASAF116'] 1990 1998
DOWNLOADING: ['CUUR0000SAF116', 'CUURS37BSAF116', 'CUURS37ASAF116'] 1999 2007
DOWNLOADING: ['CUUR0000SAF116', 'CUURS37BSAF116', 'CUURS37ASAF116'] 2008 2016
DOWNLOADING: ['CUUR0000SAF116', 'CUURS37BSAF116', 'CUURS37ASAF116'] 2017 2022


In [183]:
from functools import reduce

In [184]:
from itertools import chain

In [212]:
raw_series = [x['Results']['series'] for x in raw_responses]

In [213]:
#raw_series = list(chain(*raw_series))

In [203]:
raw_series

[[{'seriesID': 'CUUR0000SAF116',
   'data': [{'year': '1998',
     'period': 'M12',
     'periodName': 'December',
     'value': '167.2',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M11',
     'periodName': 'November',
     'value': '166.8',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M10',
     'periodName': 'October',
     'value': '166.6',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M09',
     'periodName': 'September',
     'value': '166.3',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M08',
     'periodName': 'August',
     'value': '165.7',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M07',
     'periodName': 'July',
     'value': '165.6',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M06',
     'periodName': 'June',
     'value': '165.5',
     'footnotes': [{}]},
    {'year': '1998',
     'period': 'M05',
     'periodName': 'May',
     'value': '165.2',
     'footnotes': [{}]}

* TODO: fix this big json mess up into a nice dataframe

In [226]:
series = [pd.Series(name=s['seriesID'],
                    data=[x['value'] for x in s['data']],
                    index=[date(int(x['year']), int(x['period'][1:]), 1) for x in s['data']],
                    dtype=float)
          for s in raw_series[0]]

In [ ]:
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

In [230]:
!pwd

/Users/michaelhinds/Documents/APDS/session_15Sep2022/texas_beer/Prices
